In [3]:
from pynput import mouse
import os
import settings
import cv2
import time
import pyautogui
import numpy as np
import datetime
from PIL import Image
# import ctypes
# PROCESS_PER_MONITOR_DPI_AWARE = 2
# ctypes.windll.shcore.SetProcessDpiAwareness(PROCESS_PER_MONITOR_DPI_AWARE)

left_up_x, left_up_y, right_down_x, right_down_y = [0,0,0,0]
num_press = 0

def main():
    # スクリーンショットの範囲を決定する
    left, up, width, height = decide_region()
    # スクリーンショット保存用ディレクトリを作成
    os.makedirs("Screenshot", exist_ok=True)
    # テンプレートマッチング用の画像作る
    gameset_gray, gameset_mask = make_template("Image/gameset_gray.png", "Image/gameset_mask.png")
    
    st = time.time()
    # 範囲の指定が終わったら1秒毎にスクリーンショットを撮る
    while True:
        try:
            time.sleep(0.2)
            print(time.time()-st)
            st = time.time()
            # 指定した範囲でスクリーンショットを撮影
            im = pyautogui.screenshot(region=(left, up, width, height))
#             im = Image.open("Screenshot/tmp1.png")
            # スクリーンショットをとった時間を記録
            ss_time = datetime.datetime.now().strftime("%y%m%d%H%M%S")
            path = "Screenshot/" + ss_time
            # 撮影したスクリーンショットをグレースケールに変換
            im_gray = im.convert("L")
            # OpenCVで扱えるようにImage型からndarrayに変換
            im_gray = np.array(im_gray)
            # 画像をリサイズ
            im_resize = cv2.resize(im_gray, dsize=(settings.DSIZE_W, settings.DSIZE_H))
            
            # このフレームが対戦終了タイミングかどうか
            if is_finish(im_resize, gameset_gray, gameset_mask):
#             if is_finish(cv2.resize(cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR), dsize=(settings.DSIZE_W, settings.DSIZE_H)), gameset_gray):
                # スクリーンショットを保存
                cv2.imwrite(f"{path}_finish.png", cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR))
                print("対戦終了")
                # 5秒ストップして次のスクリーンショットへ
                time.sleep(5)
                continue
            
            # Canny法でエッジを検出
            im_edges = cv2.Canny(im_resize, settings.CANNY_THRESHOLD1, settings.CANNY_THRESHOLD2, L2gradient=True)
            # Hough変換で直線を検出
            im_lines = cv2.HoughLines(im_edges, rho=1, theta=np.pi/180, threshold=settings.HOUGH_THRESHOLD)
            # このフレームが対戦開始タイミングかどうか
            if is_start(im_lines):
                # スクリーンショットを保存
                # 画像を保存するパスは "Screenshot/年月日時分秒.png"
                cv2.imwrite(f"{path}_start.png", cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR))
                print("対戦開始")
                # 5秒ストップして次のスクリーンショット
                time.sleep(5)
                continue
            # 対戦開始でも終了でもない画像を保存
            cv2.imwrite(f"{path}.png", cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR))
            # 実験中は繰り返し一回で終わり
#             break
        # プログラムが停止されたら終わり
        except KeyboardInterrupt:
            break
            
    print("end")
    
# スクリーンショット範囲の左上と右下をクリックで決定する
def decide_region():
    # マウスのクリックを検知(クリック2回目が左上，3回目が右下でスクショ範囲を指定)
    with mouse.Listener(on_click=on_click) as listener:
        listener.join()
    # スクリーンショット範囲の計算(2倍したらいい理由は謎)
    left = left_up_x*2
    up = left_up_y*2
    width = (right_down_x-left_up_x)*2
    height = (right_down_y-left_up_y)*2
    return left, up, width, height

# マウスをクリックしたときの処理
def on_click(x, y, button, pressed):
    global left_up_x, left_up_y, right_down_x, right_down_y, num_press
    if pressed:
        # 1回目のクリックはウィンドウの切り替え用で何もしない
        if num_press == 0:
            num_press = 1
        # 2回目のクリックはスクリーンショットを撮る範囲の左上を指定
        elif num_press == 1:
            num_press = 2
            left_up_x, left_up_y = x, y
            return (x, y)
        # 3回目のクリックはスクリーンショットを撮る範囲の右下を指定して終了
        else:
            right_down_x, right_down_y = x, y
            return False

# テンプレートマッチング用の画像(テンプレートとマスク)を作る
def make_template(template_path, mask_path):
    # 引数で与えられたパスからグレースケールで画像を読み込んでリサイズ
    template_gray = cv2.resize(cv2.imread(template_path, cv2.IMREAD_GRAYSCALE), dsize=(settings.DSIZE_W, settings.DSIZE_H))
    template_mask = cv2.resize(cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE), dsize=(settings.DSIZE_W, settings.DSIZE_H))
    # 128を閾値としてマスク画像を二値化
    _, template_mask = cv2.threshold(template_mask, 128, 255, cv2.THRESH_BINARY)
    # 画像の周り1/10省いてちょっとずれてもマッチングできるようにする
    template_gray = template_gray[int(settings.DSIZE_H/20):int(-settings.DSIZE_H/20), int(settings.DSIZE_W/20):int(-settings.DSIZE_W/20)]
    template_mask = template_mask[int(settings.DSIZE_H/20):int(-settings.DSIZE_H/20), int(settings.DSIZE_W/20):int(-settings.DSIZE_W/20)]
    return template_gray, template_mask
        
# 対戦終了のタイミングならTrue
def is_finish(im_gray, gameset_gray, gameset_mask):
    res = cv2.matchTemplate(im_gray, gameset_gray, cv2.TM_CCOEFF_NORMED, gameset_mask)
    _, max_val, _, _ = cv2.minMaxLoc(res)
    is_fin = True if max_val > 0.37 else False
    if is_fin:
        print(f"max_val: {max_val}")
    return is_fin

# 対戦開始のタイミングならTrue
def is_start(im_lines, error=10**-2):
    # 直線が検出されたときだけ，このフレームが対戦開始タイミングかどうかを判定する
    if im_lines is not None:
        # いい感じの直線の数
        line_count = 0
        # 画面の上部かどうかを判断する閾値
        threshold = settings.DSIZE_H*0.2
        for line in im_lines:
            line = line[0]
            # 直線が画面の上部かどうか(上部でないなら次の線)
            if line[0] > threshold:
                continue
            # いい感じの角度の直線ならカウントする
            if np.abs(line[1] - settings.STARTING_RADIAN) < error:
                line_count += 1
                # 2本あれば対戦開始のタイミング
                if line_count == 2:
                    return True
    return False

# Hough変換で求めたθとρをもとに直線を引く
def draw_line(im, theta, rho):
    h, w = im.shape[:2]
    # 直線が垂直のとき
    if np.isclose(np.sin(theta), 0):
        x1, y1 = rho, 0
        x2, y2 = rho, h
    # 直線が垂直じゃないとき
    else:
        # 直線の式を式変形すればcalc_yの式がもとまる．
        calc_y = lambda x: rho / np.sin(theta) - x * np.cos(theta) / np.sin(theta)
        x1, y1 = 0, calc_y(0)
        x2, y2 = w, calc_y(w)
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    # 直線を描画 (imにアルファチャンネルがあると黒い線分しか引かれなくなる)
    cv2.line(im, (x1, y1), (x2, y2), (0, 255, 0), 2)

if __name__ == "__main__":
    main()

0.2030956745147705
1.1660428047180176
1.0160272121429443
0.6731681823730469
0.6138880252838135
0.6187641620635986
0.6353108882904053
0.6098060607910156
0.68082594871521
0.8678188323974609
対戦開始
6.108996868133545
1.138132095336914
1.152453899383545
1.1633920669555664
1.180156946182251
1.20371413230896
1.1024608612060547
対戦終了
5.9977898597717285
1.1998751163482666
1.2017011642456055
1.1137032508850098
1.160377025604248
1.1554477214813232
0.6654329299926758
0.6327228546142578
0.6267960071563721
0.6212201118469238
0.6296029090881348
0.6349952220916748
0.6605298519134521
0.801922082901001
対戦開始
6.158582925796509
1.2064411640167236
1.2311007976531982
1.177297830581665
1.112455129623413
1.1977448463439941
1.2639448642730713
1.185302972793579
1.1236059665679932
1.2050790786743164
1.126702070236206
1.1297399997711182
1.1305248737335205
対戦終了
6.183716058731079
0.6174349784851074
0.5978538990020752
0.628612756729126
0.6246321201324463
0.6376500129699707
0.7571811676025391
0.8577070236206055
対戦開始
6.14

In [42]:
[[[ 82.          1.5358897]]

 [[100.          1.5358897]]]

SyntaxError: invalid syntax (1536401523.py, line 1)